In [ ]:
# normal imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# import codipack
import sys
sys.path.append("codiswig")
from codi import RealForward, RealReverse

In [ ]:
# define ODE solver in pure python
def mysolve(f, interv, x0, dt=0.000001):
    x = x0
    t = interv[0]
    while(t < interv[1]):
        thistime_dt = min(dt, interv[1]-t)
        x += thistime_dt * f(t,x)
        t += thistime_dt
    return x

Consider the parametric ODE $\dot y_c(t) = c y_c(t)$, $y_c(0) = 1$. The solution is $y_c(t) = \exp (ct)$. Analytically we compute $$ \tfrac\partial{\partial c} y_c(t = 2) = \tfrac\partial{\partial c} \exp(c\cdot 2) = 2 \exp(2c). $$ Let us do it with AD:

In [ ]:
# activate taping and declare c as input
RealReverse.setActive()
c = RealReverse(7.0)
c.registerInput()

# define problem
def f(t,y):
    return c*y
y0 = 1.0

# perform the algorithm
sol = mysolve(f, [0,2], y0)

# output value of solution
print("Numerical value for c="+str(c)+", t=2: ", sol)
print("Analytical value would be: ", np.exp(c*2))

In [ ]:
# declare outputs and stop taping
sol.registerOutput()
RealReverse.setPassive()
sol.setGradient(1.0)
RealReverse.evaluate()

# output derivative of solution
print("AD derivative is: ", c.getGradient())
print("Analytical derivative would be:", 2*np.exp(2*c))